In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import warnings
warnings.filterwarnings(action='ignore')

### This notebook perform cropping images that contain "Giraffe","Elephant", "Grants Gazelle". It will pick up classes "Giraffe","Elephant", "Grants Gazelle" then crop those images

In [14]:
!rm -r crop/*

In [3]:
imbalance_train=pd.read_csv("imbalance_train.csv")

In [4]:
imbalance_train.shape

(3688, 17)

In [5]:
im_df=imbalance_train[imbalance_train['labelclass_name'].isin(["Giraffe","Elephant", "Grants Gazelle"])]

In [6]:
im_df.shape

(66, 17)

In [7]:
def _crop(df,x1,y1,x2,y2,name='1',dr1=''):
    """
    df: dataframe that contain annotation
    x1, y1: relative coordinates of top-left corner(on width and height of original image) of new cropped image
    x2, y2: relative coordinates of bottom-right corner(on width and height of original image) of new cropped image
    x1,y1,x2,y2 are between 0 and 1
    """
    dr='/home/jovyan/lost+found/annotations/'
    #filter objects in cropped region
    df1=df[(df['x_c'] > x1*df['im_width']) \
           & (df['x_c'] <x2*df['im_width']) \
           & (df['y_c'] > y1*df['im_height']) \
           & (df['y_c'] < y2*df['im_height'])].copy()
    
    #crop images and update filepath
    df1_path=np.unique(df1['filepath'])
    for f in df1_path:
        im_df=df1[df1['filepath']==f].copy()
        w=np.unique(im_df['im_width'])
        h=np.unique(im_df['im_height'])
        im=Image.open(f)
        im1=im.crop((int(x1*w),int(y1*h),int(x2*w),int(y2*h)))
        im1.save(dr1+name+f.replace("/","_"))
        df1.loc[df1['filepath']==f,'filepath'] = dr+dr1+ name + f.replace("/","_")
     
    #update object coordinates
    df1.x_min=round(df1.x_min-x1*df1.im_width)
    df1.x_max=round(df1.x_max-x1*df1.im_width)
    df1.y_min=round(df1.y_min-y1*df1.im_height)
    df1.y_max=round(df1.y_max-y1*df1.im_height)
    
    #update new size for cropped images
    df1.im_width=round((x2-x1)*df1.im_width)
    df1.im_height=round((y2-y1)*df1.im_height)
    
    #correct location of objects:
    df1.loc[df1['x_min']<0,'x_min']=0 #ensure bounding boxes are within cropped images
    df1.loc[df1['y_min']<0,'y_min']=0 #ensure bounding boxes are within cropped images
    df1['x_max']=df1[['x_max','im_width']].min(axis=1) #ensure bounding boxes are within cropped images
    df1['y_max']=df1[['y_max','im_height']].min(axis=1) #ensure bounding boxes are within cropped images
    
    #update centers of objects
    df1.x_c=round(0.5*(df1.x_min+df1.x_max))
    df1.y_c=round(0.5*(df1.y_min+df1.y_max))
    df1.width=round((df1.x_max-df1.x_min)/df1.im_width,3)
    df1.height=round((df1.y_max-df1.y_min)/df1.im_height,3)
    
    return df1.copy()
    

In [8]:
Imbalance_crop=imbalance_train.copy()
for i in range(0,8):
    crop_i=_crop(im_df,i/20, 0, 1, 1,name=str(i),dr1="crop/")
    crop_j=_crop(im_df,0, 0, 1-1.2*(8-i)/20, 1,name=str(i+10),dr1="crop/")
    crop_ij=_crop(im_df,i/17,i/25 , 1, 1,name=str(i+20),dr1="crop/")
    crop_ji=_crop(im_df,0, 0, 1-(8-i)/20, 1-(8-i)/25,name=str(i+30),dr1="crop/")
    crop_ii=_crop(im_df,i/30,i/35 , 1-(8-i)/30, 1-(8-i)/35,name=str(i+40),dr1="crop/")
    crop_jj=_crop(im_df,i/20,i/20 , i/20+0.5, i/20+0.5,name=str(i+50),dr1="crop/")
    Imbalance_crop=Imbalance_crop.append([crop_i.copy(),crop_j.copy(),crop_ij.copy(),crop_ji.copy(),
                                          crop_ii.copy(),crop_jj.copy()],ignore_index=False)

In [9]:
Imbalance_crop.shape

(5787, 17)

In [10]:
def img_count(dat):
    #Count how many images for each type of class:
    count={}
    for lb in np.unique(dat['labelclass_name']):
        df=dat[dat['labelclass_name']==lb]
        count[lb]=len(np.unique(df['filepath']))

    return count

In [11]:
img_count(Imbalance_crop)

{'Elephant': 373, 'Giraffe': 419, 'Grants Gazelle': 360, 'Wildebeest': 389}

In [12]:
Imbalance_crop.to_csv("imbalance_crop.csv",index=False)

In [70]:
pwd

'/home/jovyan/lost+found/annotations'

In [13]:
!cp '/home/jovyan/lost+found/annotations/imbalance_crop.csv' '/home/jovyan/lost+found/yolo3/train/imbalance_crop.csv'